In [ ]:
from sklearn.cluster import KMeans
import sklearn.metrics as sm
import numpy as np

In [ ]:
import pandas as pd
import pandas_gbq
import io
import datetime as dt
from functools import reduce
import requests
import time
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('/work/key.json')
bq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [ ]:
mdp_data= pd.io.gbq.read_gbq('''
select
    distinct cast(a.item_id as int64) as ID,
    a.view_item_list_1d,
    view_item_list_7d,
    a.view_item_list_30d,
    a.view_item_1d,
    a.view_item_7d,
    a.view_item_30d,
    a.add_to_cart_1d,
    a.add_to_cart_7d,
    a.add_to_cart_30d,
    a.begin_checkout_1d,
    a.begin_checkout_7d,
    a.begin_checkout_30d,
    a.purchase_1d,
    a.purchase_7d,
    a.purchase_30d,
    a.qty_sold,
    a.nbre_commande,
    a.product_marge,
    a.order_recency,
    a.order_seniority,
    cast(a.stock as int64) as stock,
from `mdp-paris.dbt_pipeline_prod.con_product_segmentation` a
left join
    `mdp-paris.dbt_pipeline_prod.stg_presta_product_performance` b
    on a.item_id = b.product_id
where collection = 'Collection 2022'

''', project_id=credentials.project_id, dialect='standard', credentials = credentials )

In [ ]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame)
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
df=clean_dataset(mdp_data)
df.columns

Index(['ID', 'view_item_list_1d', 'view_item_list_7d', 'view_item_list_30d',
       'view_item_1d', 'view_item_7d', 'view_item_30d', 'add_to_cart_1d',
       'add_to_cart_7d', 'add_to_cart_30d', 'begin_checkout_1d',
       'begin_checkout_7d', 'begin_checkout_30d', 'purchase_1d', 'purchase_7d',
       'purchase_30d', 'qty_sold', 'nbre_commande', 'product_marge',
       'order_recency', 'order_seniority', 'stock'],
      dtype='object')

In [ ]:
model=KMeans(n_clusters=4)
model.fit(df[['view_item_list_1d', 'view_item_list_7d', 'view_item_list_30d',
       'view_item_1d', 'view_item_7d', 'view_item_30d', 'add_to_cart_1d',
       'add_to_cart_7d', 'add_to_cart_30d', 'begin_checkout_1d',
       'begin_checkout_7d', 'begin_checkout_30d', 'purchase_1d', 'purchase_7d',
       'purchase_30d', 'qty_sold', 'nbre_commande', 'product_marge',
       'order_recency', 'order_seniority', 'stock']])

KMeans(n_clusters=4)

In [ ]:
df['Cluster']=model.labels_

In [ ]:
df[['view_item_list_1d', 'view_item_list_7d', 'view_item_list_30d',
       'view_item_1d', 'view_item_7d', 'view_item_30d', 'add_to_cart_1d',
       'add_to_cart_7d', 'add_to_cart_30d', 'begin_checkout_1d',
       'begin_checkout_7d', 'begin_checkout_30d', 'purchase_1d', 'purchase_7d',
       'purchase_30d', 'qty_sold', 'nbre_commande', 'product_marge',
       'order_recency', 'order_seniority', 'stock','Cluster']].groupby('Cluster').mean()

,view_item_list_1d,view_item_list_7d,view_item_list_30d,view_item_1d,view_item_7d,view_item_30d,add_to_cart_1d,add_to_cart_7d,add_to_cart_30d,begin_checkout_1d,...,begin_checkout_30d,purchase_1d,purchase_7d,purchase_30d,qty_sold,nbre_commande,product_marge,order_recency,order_seniority,stock
Cluster,,,,,,,,,,,,,,,,,,,,,
0,726.607843,7556.176471,22244.078431,26.862745,232.166667,849.735294,1.029412,12.382353,36.823529,1.382353,...,39.990196,0.215686,1.852941,5.990196,18.254902,17.764706,4275.264706,7.441176,123.343137,36.676471
1,2450.964286,27116.000000,73445.250000,74.571429,612.392857,1796.750000,4.571429,30.714286,82.785714,2.571429,...,84.607143,0.250000,4.107143,11.750000,31.892857,31.607143,7167.714286,3.071429,95.964286,14.500000
2,4480.900000,45861.500000,135297.700000,155.200000,1262.000000,3914.100000,5.100000,43.000000,119.900000,5.600000,...,114.100000,0.300000,6.000000,17.200000,33.000000,32.400000,8398.400000,2.100000,78.900000,6.700000
3,1309.033898,14591.847458,43800.745763,51.118644,404.016949,1835.949153,2.271186,21.237288,64.694915,2.423729,...,69.508475,0.338983,3.050847,9.542373,32.016949,31.491525,7297.050847,4.237288,142.610169,29.084746


In [ ]:
df['Cluster'].replace([0,1,2,3],['Cold','Hot Old','Top Hot','Medium Old'],inplace=True)

#rename cluster column
df.rename(columns={"Cluster": "Segments"},inplace=True)

In [ ]:
df

,ID,view_item_list_1d,view_item_list_7d,view_item_list_30d,view_item_1d,view_item_7d,view_item_30d,add_to_cart_1d,add_to_cart_7d,add_to_cart_30d,...,purchase_1d,purchase_7d,purchase_30d,qty_sold,nbre_commande,product_marge,order_recency,order_seniority,stock,Segments
1,7564.0,883.0,10210.0,28088.0,30.0,238.0,699.0,3.0,29.0,79.0,...,1.0,5.0,14.0,27.0,27.0,6183.0,1.0,170.0,248.0,Top Hot
2,7565.0,613.0,6655.0,18279.0,23.0,151.0,341.0,1.0,10.0,19.0,...,0.0,0.0,1.0,2.0,2.0,208.0,14.0,148.0,45.0,Top Hot
3,7566.0,574.0,6680.0,17316.0,14.0,121.0,265.0,1.0,16.0,31.0,...,0.0,1.0,3.0,9.0,4.0,416.0,2.0,84.0,60.0,Top Hot
4,7567.0,1214.0,13127.0,35223.0,45.0,377.0,1712.0,0.0,13.0,37.0,...,0.0,2.0,5.0,20.0,20.0,4760.0,2.0,171.0,3.0,Cold
7,7570.0,664.0,7270.0,19793.0,14.0,165.0,661.0,0.0,13.0,44.0,...,0.0,2.0,5.0,34.0,34.0,7786.0,5.0,170.0,5.0,Top Hot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,8409.0,848.0,7541.0,25577.0,30.0,233.0,626.0,0.0,10.0,24.0,...,0.0,2.0,5.0,8.0,8.0,968.0,4.0,18.0,31.0,Top Hot
277,8429.0,984.0,8378.0,30722.0,41.0,337.0,1414.0,2.0,9.0,34.0,...,0.0,3.0,4.0,4.0,4.0,816.0,3.0,8.0,89.0,Top Hot
278,8431.0,1026.0,9523.0,34023.0,69.0,534.0,2232.0,1.0,7.0,15.0,...,0.0,1.0,1.0,1.0,1.0,204.0,2.0,2.0,29.0,Top Hot
279,8433.0,1011.0,8751.0,31753.0,35.0,266.0,918.0,0.0,3.0,14.0,...,0.0,1.0,1.0,2.0,2.0,476.0,4.0,9.0,101.0,Top Hot


In [ ]:
base_image=pd.read_csv('base_image.csv',sep=',')

In [ ]:
final=df.merge(base_image[['ID','URL de l\'image 1']],how='left',on='ID')
final=final.rename(columns={"URL de l\'image 1": "image"})
final

,ID,view_item_list_1d,view_item_list_7d,view_item_list_30d,view_item_1d,view_item_7d,view_item_30d,add_to_cart_1d,add_to_cart_7d,add_to_cart_30d,...,purchase_7d,purchase_30d,qty_sold,nbre_commande,product_marge,order_recency,order_seniority,stock,Segments,image
0,7564.0,883.0,10210.0,28088.0,30.0,238.0,699.0,3.0,29.0,79.0,...,5.0,14.0,27.0,27.0,6183.0,1.0,170.0,248.0,Top Hot,https://media.mesdemoisellesparis.com/20553-fa...
1,7565.0,613.0,6655.0,18279.0,23.0,151.0,341.0,1.0,10.0,19.0,...,0.0,1.0,2.0,2.0,208.0,14.0,148.0,45.0,Top Hot,https://media.mesdemoisellesparis.com/20588-fa...
2,7566.0,574.0,6680.0,17316.0,14.0,121.0,265.0,1.0,16.0,31.0,...,1.0,3.0,9.0,4.0,416.0,2.0,84.0,60.0,Top Hot,https://media.mesdemoisellesparis.com/20653-fa...
3,7567.0,1214.0,13127.0,35223.0,45.0,377.0,1712.0,0.0,13.0,37.0,...,2.0,5.0,20.0,20.0,4760.0,2.0,171.0,3.0,Cold,https://media.mesdemoisellesparis.com/20552-fa...
4,7570.0,664.0,7270.0,19793.0,14.0,165.0,661.0,0.0,13.0,44.0,...,2.0,5.0,34.0,34.0,7786.0,5.0,170.0,5.0,Top Hot,https://media.mesdemoisellesparis.com/20543-fa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,8409.0,848.0,7541.0,25577.0,30.0,233.0,626.0,0.0,10.0,24.0,...,2.0,5.0,8.0,8.0,968.0,4.0,18.0,31.0,Top Hot,NaN
195,8429.0,984.0,8378.0,30722.0,41.0,337.0,1414.0,2.0,9.0,34.0,...,3.0,4.0,4.0,4.0,816.0,3.0,8.0,89.0,Top Hot,NaN
196,8431.0,1026.0,9523.0,34023.0,69.0,534.0,2232.0,1.0,7.0,15.0,...,1.0,1.0,1.0,1.0,204.0,2.0,2.0,29.0,Top Hot,NaN
197,8433.0,1011.0,8751.0,31753.0,35.0,266.0,918.0,0.0,3.0,14.0,...,1.0,1.0,2.0,2.0,476.0,4.0,9.0,101.0,Top Hot,NaN


In [ ]:
mdp_ref= pd.io.gbq.read_gbq('''
select
    distinct cast(a.item_id as int64) as ID,
    b.reference
from `mdp-paris.dbt_pipeline_prod.con_product_segmentation` a
left join
    `mdp-paris.dbt_pipeline_prod.stg_presta_product_performance` b
    on a.item_id = b.product_id
where collection = 'Collection 2022'

''', project_id=credentials.project_id, dialect='standard', credentials = credentials )

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=08a4d91e-cdf8-4f34-a2e4-ebb5990d58fe' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>